In [1]:
#!pip install fastapi
#!pip install uvicorn

In [2]:
import sqlite3
def init_table():
    db = sqlite3.connect('user.db')
    cursor = db.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS user(id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT NOT NULL UNIQUE)
    ''')
    db.commit()
    db.close()
# init_table()

In [3]:
from pydantic import BaseModel
class userModel(BaseModel):
    name: str 
    
def connection():
    db = sqlite3.connect('user.db')
    if db is not None:
        return db
    else:
        print('error')
        return None

def insert_user(user: userModel):
    db = connection()
    cursor = db.cursor()
    cursor.execute('''
        INSERT INTO user(name) VALUES(?)
    ''',(user.name,))
    db.commit()
    cursor.execute('''
            SELECT * FROM user WHERE id = last_insert_rowid()
        ''')
    last_user = cursor.fetchone()    
    db.close()
    return last_user

def update_user(user: userModel, id: int):
    db = connection()
    cursor = db.cursor()
    cursor.execute('''
        UPDATE user SET name = ? WHERE id = ?
    ''',(user.name, id))
    db.commit()
    db.close()
    return {'name': user.name}

def delete_user(id: int):
    db = connection()
    cursor = db.cursor()
    cursor.execute('''
        DELETE FROM user WHERE id = ?
    ''',(id,))
    db.commit()
    db.close()
    

def get_user():
    db = connection()
    cursor = db.cursor()
    cursor.execute('''
        SELECT * FROM user ORDER BY id DESC
    ''')
    all = cursor.fetchall()
    db.close()
    return all

def get_user_id(id: int):
    db = connection()
    cursor = db.cursor()
    cursor.execute('''
        SELECT * FROM user WHERE id = ?
    ''',(id,))
    all = cursor.fetchone()
    db.close()
    return all
    

In [4]:
from typing import  Union
from fastapi import FastAPI, HTTPException
import asyncio,uvicorn
app = FastAPI()

@app.get("/")
def read_root():
    return {"Hello": "World"}

@app.get("/user")
def read_user():
    return get_user();

@app.get("/table")
def read_table():
    return init_table();

@app.get("/user/{id}")
def get_user_by_id(id: int):
    return get_user_id(id)


@app.post("/user")
def create_user(user: userModel):
    return insert_user(user)

@app.put("/user/{id}")
def update_users(id: int, user: userModel):
    return update_user(user, id)

@app.delete("/user/{id}")
def delete_users(id: int):
    return delete_user(id)


In [5]:
if __name__ == "__main__":
    # set host reload
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [6357]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:54142 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:54153 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:54153 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:54164 - "POST /user HTTP/1.1" 200 OK
INFO:     127.0.0.1:54183 - "GET /table HTTP/1.1" 200 OK
INFO:     127.0.0.1:54200 - "GET /user/1 HTTP/1.1" 200 OK
INFO:     127.0.0.1:54233 - "POST /user HTTP/1.1" 200 OK
INFO:     127.0.0.1:54254 - "PUT /user/2 HTTP/1.1" 200 OK
INFO:     127.0.0.1:54265 - "DELETE /user/2 HTTP/1.1" 200 OK
INFO:     127.0.0.1:54276 - "GET /user HTTP/1.1" 200 OK
INFO:     127.0.0.1:54608 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:54608 - "GET /user HTTP/1.1" 200 OK
INFO:     127.0.0.1:55275 - "GET /user HTTP/1.1" 200 OK
INFO:     127.0.0.1:55276 - "GET /user HTTP/1.1" 200 OK
INFO:     127.0.0.1:55278 - "GET /user HTTP/1.1" 200 OK
INFO:     127.0.0.1:55282 - "GET /user HTTP/1.1" 200 OK
INFO:     127.0.0.1:55283 - "GET /user HTTP/1.1" 200 OK
INFO:     127.0.0.1:55339 - "POST /u